# Lineare Abbildungen - Interaktive Grafiken

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import imageio

In [2]:
# Global Variables
fps     = 15
figsize = (2.2,1.7)
dpi     = 150

dotted = {'linestyle':'dashed', 'color':'black', 'linewidth':0.5}
axis = {'fc':'k', 'ec':'k', 'head_width':0.05,  'head_length':0.1, 'clip_on':False}

## Streckungen

### Allgemeines Beispiel

In [215]:
class Arrow():
    def __init__(self, ori_over_time, vector_over_time, alpha_over_time=1, text='', text_shift=[0,0], color='black'):
        self.looks_dict = {'head_width':0.05, 'head_length':0.1, 'fc':color, 'ec':color, 'length_includes_head':True}
        if (len(np.array(ori_over_time).shape) != 2) or (len(np.array(vector_over_time).shape) != 2):
            raise Exception('Please provide 2D coordinates over time (first dim)')
        self.ori_over_time = ori_over_time; self.vector_over_time = vector_over_time
        self.alpha_over_time = alpha_over_time
        self.color = color
        self.text_coordinates_over_time = np.array(ori_over_time) + 0.5*np.array(vector_over_time) + np.array(text_shift)
        self.text = text

class Animation():
    def __init__(self, n_frames=80, dpi=150, fps=15, figsize=(2.2,1.7), size=[0, 2.5, 0, 2.5]):
        self.dpi=dpi; self.fps=fps; self.figsize=figsize; self.n_frames=n_frames
        self.size=size
        self.arrows = []
        self.texts = []
    
    def create_fractional_timing(self, zeros, ramp, ones):
        fractional_timing = np.array([0]*zeros + list(np.linspace(0,1,ramp)) + [1]*ones)
        if len(fractional_timing) != self.n_frames:
            raise Exception('Input is inconsistent with number of frames in animation!')
        return fractional_timing
    
    def create_axis(self, ax):
        axis = {'fc':'k', 'ec':'k', 'head_width':0.05,  'head_length':0.1, 'clip_on':False, 'length_includes_head':True}
        for side in ['bottom','right','top','left']:
            ax.spines[side].set_visible(False)
        ax.arrow(self.size[0], 0, self.size[1], 0, **axis);
        ax.arrow(0, self.size[2], 0, self.size[3], **axis);
        ax.set_xticks([]); ax.set_yticks([]);
        
    def format_plot(self, ax):
        self.create_axis(ax)
        ax.set_xlim(self.size[0], self.size[1])
        ax.set_ylim(self.size[2], self.size[3])
        
    def add_arrows(self, arrows):
        self.arrows.extend(arrows)
        
    def render_image(self, output='hallo.gif'):
        with imageio.get_writer(output, mode='I', fps=self.fps) as writer:
            for frame in np.arange(self.n_frames):
                fig, ax = plt.subplots(figsize=self.figsize, dpi=self.dpi)
                
                for arrow in self.arrows:
                    ori = arrow.ori_over_time[frame]; vec = arrow.vector_over_time[frame]
                    text_coord = arrow.text_coordinates_over_time[frame]
                    ax.arrow(ori[0],ori[1],vec[0],vec[1], **arrow.looks_dict)
                    ax.text(text_coord[0], text_coord[1], arrow.text)
                
                self.format_plot(ax)
                
                plt.savefig('tmp.png');
                plt.close(fig);
                image = imageio.imread('tmp.png');
                writer.append_data(image);
                
ani = Animation(n_frames=30)

t1 = ani.create_fractional_timing(10,10,10)
I = np.ones_like(t1)

arr1 = Arrow(np.array([I*0,I*0]).T, np.array([t1*1+1,I*0+1]).T, I, color='red', text=r'$f(a)$')
arr2 = Arrow(np.array([I*0,I*0]).T, np.array([I*1+1,t1*1+1]).T, I, color='blue', text=r'$f(a)$')
ani.add_arrows([arr1,arr2])
ani.render_image('test.gif')



In [207]:
a = []

In [208]:
a.extend([0,0])

In [209]:
a

[0, 0]

### Streckung entlang einer Achse

In [13]:
with imageio.get_writer('streckung1.gif', mode='I', fps=fps) as writer:
    timeaxis = np.linspace(1,2,20)
    timeaxis = np.pad(timeaxis, 10, mode='edge')
    for t in timeaxis:
        t_norm = (t-timeaxis.min())/(timeaxis.max()-timeaxis.min())
        fig, ax = plt.subplots(figsize=figsize, dpi=dpi);
        ### Actual Plot #############################################################
        for side in ['bottom','right','top','left']:
            ax.spines[side].set_visible(False)
        ax.arrow(0, 0, 2.6, 0, **axis);
        ax.arrow(0, 0, 0, 2.6, **axis);
        plt.xticks([]); plt.yticks([]);
        
        vec1 = np.array([1*t, 2])
        ax.arrow(0, 0, vec1[0],vec1[1], head_width=0.05, head_length=0.1, \
                 fc='red', ec='red', length_includes_head=True)
        #ax.text(vec1[0],vec1[1],'(%.1f,%.1f)ᵀ'%(vec1[0],vec1[1]), fontsize=8)
        ax.text(vec1[0]/2-0.2,vec1[1]/2+0.25,  r'$a$', fontsize=8, 
                color='red', alpha = (1-t_norm)**6)
        ax.text(vec1[0]/2-0.2,vec1[1]/2+0.25,  r'$f(a)$', 
                fontsize=8, color='red', alpha = t_norm**6)
        
        #ax.text(0.25,2.25, r'$\beta=%.1f$'%t, fontsize=8, color='black')
        
        ax.axvline(1., **dotted); 
        ax.axvline(2, **dotted); 
        ax.axhline(2, **dotted);
        ax.set_xlim(0,2.5); ax.set_ylim(0,2.5)
        #############################################################################
        plt.savefig('tmp.png');
        plt.close(fig);
        image = imageio.imread('tmp.png');
        writer.append_data(image);

### Streckung einer Summe zweier Vektoren, Additivitaet

In [19]:
with imageio.get_writer('streckungSumme.gif', mode='I', fps=fps) as writer: 
    timeaxis = np.linspace(1,2,20)
    timeaxis = np.pad(timeaxis, 10, mode='edge')
    for t in timeaxis:
        t_norm = (t-timeaxis.min())/(timeaxis.max()-timeaxis.min())
        fig, ax = plt.subplots(figsize=figsize, dpi=dpi);
        ### Actual Plot #############################################################
        for side in ['bottom','right','top','left']:
            ax.spines[side].set_visible(False)
        ax.arrow(-2.6, 0, 2*2.6, 0, **axis); #x
        ax.arrow(0, 0, 0, 2.6, **axis); #y
        ax.spines['left'].set_position('center')
        plt.xticks([]); plt.yticks([]);
        
        vec1 = np.array([-1*t, 0.5])
        vec2 = np.array([2*t, 1.5])
        summe = vec1+vec2
        ax.arrow(0, 0, vec1[0],vec1[1], head_width=0.05, head_length=0.1, \
                 fc='green', ec='green', length_includes_head=True)
        ax.arrow(vec1[0],vec1[1], vec2[0],vec2[1], head_width=0.05, head_length=0.1, \
                 fc='blue', ec='blue', length_includes_head=True)
        ax.arrow(0,0, summe[0],summe[1], head_width=0.05, head_length=0.1, \
                 fc='red', ec='red', length_includes_head=True, linewidth=1.5)
        
        ax.text(vec1[0]/2,vec1[1]/2+0.05, r'$a$', fontsize=8, 
                color='green', alpha = (1-t_norm)**6)
        ax.text(vec1[0]+vec2[0]/4-0.1,vec1[1]+vec2[1]/4+0.3, r'$b$', 
                fontsize=8, color='blue', alpha = (1-t_norm)**6)
        ax.text(summe[0]/2,summe[1]/2-0.2, r'$a+b$', fontsize=8, 
                color='red', alpha = (1-t_norm)**6)
        
        ax.text(vec1[0]/2,vec1[1]/2+0.05, r'$f(a)$', fontsize=8, 
                color='green', alpha = t_norm**6)
        ax.text(vec1[0]+vec2[0]/4-0.1,vec1[1]+vec2[1]/4+0.3, r'$f(b)$', 
                fontsize=8, color='blue', alpha = t_norm**6)
        ax.text(summe[0]/2,summe[1]/2-0.2, r'$f(a+b)$', fontsize=8, 
                color='red', alpha = t_norm**6)
        
        #ax.text(-1.5,2.25, r'$\beta=%.1f$'%t, fontsize=8, color='black')
        
        for i in [1,2, -1, -2]:
            ax.axvline(i, **dotted); 
        for i in [2,0.5]:
            ax.axhline(i, **dotted);
        ax.set_xlim(-2.5,2.5); ax.set_ylim(0,2.5)
        #############################################################################
        plt.savefig('tmp.png');
        plt.close(fig);
        image = imageio.imread('tmp.png');
        writer.append_data(image);

### Skalierung eines Vektors, Homogenitaet

In [15]:
with imageio.get_writer('streckung_homogenitaet.gif', mode='I', fps=fps) as writer:
    var1 = np.array([0]*10 + list(np.linspace(0,1,20)) + [1]*60)
    var2 = np.array([0]*40 + list(np.linspace(0,1,20)) + [1]*30)
    for v1,v2 in zip(var1,var2):
        fig, ax = plt.subplots(figsize=figsize, dpi=dpi);
        ### Actual Plot #############################################################
        for side in ['bottom','right','top','left']:
            ax.spines[side].set_visible(False)
        ax.arrow(0, 0, 2.6, 0, **axis);
        ax.arrow(0, 0, 0, 2.6, **axis);
        plt.xticks([]); plt.yticks([]);
        
        for i in [0.4, 0.8, 1.2, 2.4]:
            ax.axvline(i, **dotted); 
        for i in [0.8, 2.4]:
            ax.axhline(i, **dotted);
        
        lam_begin = 1
        lam_end   = 3
        
        beta_begin = 1
        beta_end   = 2
        
        vec_ori = np.array([0.4, 0.8])
        vec1 = vec_ori * (lam_begin + v1*(lam_end-lam_begin)) * \
                np.array([(beta_begin + v2*(beta_end-beta_begin)),1]) 
        vec2 = vec_ori * (lam_begin + v2*(lam_end-lam_begin)) * \
                np.array([(beta_begin + v1*(beta_end-beta_begin)),1]) 

        # Vector 1
        ax.arrow(0, 0, vec1[0], vec1[1], head_width=0.05, head_length=0.1, \
                 fc='red', ec='red', length_includes_head=True)
        ax.text(vec1[0]/2-0.25,vec1[1]/2+0.25,  r'$\lambda \cdot a$', 
            fontsize=8, color='red', alpha = (v1-v2)**6)
        
        # Vector 2
        ax.arrow(0, 0, vec2[0], vec2[1], head_width=0.05, head_length=0.1, \
                 fc='blue', ec='blue', length_includes_head=True)
        ax.arrow(0, 0, vec2[0], vec2[1], head_width=0.05, head_length=0.1, \
                 fc='k', ec='k', length_includes_head=True, alpha=v2**10)
        ax.text(vec1[0]/2-0.1,vec1[1]/2+0.25,  r'$a$', fontsize=8, 
                color='black', alpha = (1-v1)**6)
        ax.text(vec2[0]/2+0.2,vec2[1]/2+0.0,  r'$f(a)$', 
                fontsize=8, color='blue', alpha = (v1-v2)**6)
        ax.text(vec1[0]/2-0.1,vec1[1]/2+0.25,  r'$a$', fontsize=8, 
                color='black', alpha = (1-v1)**6)
        ax.text(vec1[0]/2+0.1,vec1[1]/2-0.1,  r'$\lambda\cdot f(a)$', fontsize=8, 
                color='blue', alpha = v2**6)
        ax.text(vec1[0]/2+0.1,vec1[1]/2+0.7,  r'$f(\lambda\cdot a)$', fontsize=8, 
                color='red', alpha = v2**6)
        
        ax.set_xlim(0,2.5); ax.set_ylim(0,2.5)
        #############################################################################
        plt.savefig('tmp.png');
        plt.close(fig);
        image = imageio.imread('tmp.png');
        writer.append_data(image);